 # TLT FasterRCNN example usecase

 This notebook shows an example usecase of FasterRCNN using Transfer Learning Toolkit.

 0. [Set up env variables](#head-0)
 1. [Prepare dataset and pretrained model](#head-1)<br>
     1.1 [Download pretrained model](#head-1-1)<br>
 2. [Provide training specification](#head-2)
 3. [Run TLT training](#head-3)
 4. [Evaluate trained models](#head-4)
 5. [Prune trained models](#head-5)
 6. [Retrain pruned models](#head-6)
 7. [Evaluate retrained model](#head-7)
 8. [Visualize inferences](#head-8)
 9. [Deploy](#head-9)

 ## 0. Set up env variables <a class="anchor" id="head-0"></a>

In [ ]:
# Setting up env variables for cleaner command line commands.
print("Please replace the variables with your own.")
%env KEY=tlt
%env USER_EXPERIMENT_DIR=/workspace/tlt-experiments
%env DATA_DOWNLOAD_DIR=/workspace/tlt-experiments/data
%env SPECS_DIR=./specs
!mkdir -p $DATA_DOWNLOAD_DIR/faster_rcnn
# Prepend current directory and HOME directory to the PATH env variable.
import os
os.environ['PATH'] = './:' + os.environ.get('HOME', '') + ':' + os.environ['PATH']

 ## 1. Prepare dataset and pretrained model <a class="anchor" id="head-1"></a>

 We will be using the KITTI detection dataset for the tutorial. To find more details please visit
 http://www.cvlibs.net/datasets/kitti/eval_object.php?obj_benchmark=2d. Please download the KITTI detection images (http://www.cvlibs.net/download.php?file=data_object_image_2.zip) and labels (http://www.cvlibs.net/download.php?file=data_object_label_2.zip) to $DATA_DOWNLOAD_DIR.

In [ ]:
# Check the dataset is present
!mkdir -p $DATA_DOWNLOAD_DIR
!if [ ! -f $DATA_DOWNLOAD_DIR/data_object_image_2.zip ]; then echo 'Image zip file not found, please download.'; else echo 'Found Image zip file.';fi
!if [ ! -f $DATA_DOWNLOAD_DIR/data_object_label_2.zip ]; then echo 'Label zip file not found, please download.'; else echo 'Found Labels zip file.';fi

In [ ]:
# unpack 
!unzip -u $DATA_DOWNLOAD_DIR/data_object_image_2.zip -d $DATA_DOWNLOAD_DIR
!unzip -u $DATA_DOWNLOAD_DIR/data_object_label_2.zip -d $DATA_DOWNLOAD_DIR

In [ ]:
# verify
!ls -l $DATA_DOWNLOAD_DIR/

Additionally, if you have your own dataset already in a volume (or folder), you can mount the volume on `DATA_DOWNLOAD_DIR` (or create a soft link). Below shows an example:
```bash
# if your dataset is in /dev/sdc1
mount /dev/sdc1 $DATA_DOWNLOAD_DIR

# if your dataset is in folder /var/dataset
ln -sf /var/dataset $DATA_DOWNLOAD_DIR
```

### 1.1 Prepare tfrecords from kitti format dataset <a class="anchor" id="head-1-1"></a>

* Update the tfrecords spec file to take in your kitti format dataset
* Create the tfrecords using the tlt-dataset-convert 
* TFRecords only need to be generated once.

In [ ]:
print("TFrecords conversion spec file for training")
!cat $SPECS_DIR/frcnn_tfrecords_kitti_trainval.txt

In [ ]:
# Creating a new directory for the output tfrecords dump.
!mkdir -p $USER_EXPERIMENT_DIR/tfrecords
#KITTI trainval
!tlt-dataset-convert -d $SPECS_DIR/frcnn_tfrecords_kitti_trainval.txt \
                     -o $USER_EXPERIMENT_DIR/tfrecords/kitti_trainval/kitti_trainval

In [ ]:
!ls -rlt $USER_EXPERIMENT_DIR/tfrecords/kitti_trainval

 ### 1.2 Download pre-trained model <a class="anchor" id="head-1-1"></a>

In [ ]:
!ngc registry model list nvidia/tlt_pretrained_object_detection*

In [ ]:
# Download model from NGC.
!ngc registry model download-version nvidia/tlt_pretrained_object_detection:resnet18

In [ ]:
# Copy weights to data directory.
!cp tlt_pretrained_object_detection_vresnet18/resnet_18.hdf5 $DATA_DOWNLOAD_DIR/faster_rcnn/
!rm -rf tlt_pretrained_object_detection_vresnet18
!ls -rlt $DATA_DOWNLOAD_DIR/faster_rcnn

 ## 2. Provide training specification <a class="anchor" id="head-2"></a>

In [ ]:
!sed -i 's/$KEY/'"$KEY/g" $SPECS_DIR/default_spec_resnet18.txt
!cat $SPECS_DIR/default_spec_resnet18.txt

 ## 3. Run TLT training <a class="anchor" id="head-3"></a>
 * Provide the sample spec file for training.

In [ ]:
!tlt-train faster_rcnn -e $SPECS_DIR/default_spec_resnet18.txt

In [ ]:
print('Model for each epoch:')
print('---------------------')
!ls -lh $USER_EXPERIMENT_DIR/data/faster_rcnn

In [ ]:
print("For multi-GPU, please uncomment and run this instead. Change --gpus based on your machine.")
# !tlt-train faster_rcnn -e $SPECS_DIR/default_spec_resnet18.txt \
#                        --gpus 2

In [ ]:
print("For resume training from checkpoint, please uncomment and run this instead. Change/Add the 'resume_from_model' field in the spec file.")
# !tlt-train faster_rcnn -e $SPECS_DIR/default_spec_resnet18.txt

 ## 4. Evaluate trained models <a class="anchor" id="head-4"></a>

In [ ]:
!tlt-evaluate faster_rcnn -e $SPECS_DIR/default_spec_resnet18.txt

 ## 5. Prune trained models <a class="anchor" id="head-5"></a>
 * Specify pre-trained model
 * Equalization criterion
 * Threshold for pruning
 * A key to save and load the model
 * Output directory to store the model
 
Usually, you just need to adjust `-pth` (threshold) for accuracy and model size trade off. Higher `pth` gives you smaller model (and thus higher inference speed) but worse accuracy. The threshold to use is depend on the dataset. A pth value 0.4 is just a start point. If the retrain accuracy is good, you can increase this value to get smaller models. Otherwise, lower this value to get better accuracy.

In [ ]:
!tlt-prune -m $USER_EXPERIMENT_DIR/data/faster_rcnn/frcnn_kitti_resnet18.epoch12.tlt \
           -o $USER_EXPERIMENT_DIR/data/faster_rcnn/model_1_pruned.tlt  \
           -eq union  \
           -pth 0.4 \
           -k $KEY

In [ ]:
!ls -rlt $USER_EXPERIMENT_DIR/data/faster_rcnn

 ## 6. Retrain pruned models <a class="anchor" id="head-6"></a>
 * Model needs to be re-trained to bring back accuracy after pruning
 * Specify re-training specification

In [ ]:
# Here we have updated the spec file to include the newly pruned model as a pretrained weights.
!sed -i 's/$KEY/'"$KEY/g" $SPECS_DIR/default_spec_resnet18_retrain_spec.txt
!cat $SPECS_DIR/default_spec_resnet18_retrain_spec.txt

In [ ]:
# Retraining using the pruned model as pretrained weights 
!tlt-train faster_rcnn -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt

In [ ]:
# Listing the newly retrained model.
!ls -rlt $USER_EXPERIMENT_DIR/data/faster_rcnn

 ## 7. Evaluate retrained model <a class="anchor" id="head-7"></a>

In [ ]:
!tlt-evaluate faster_rcnn -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt

 ## 8. Visualize inferences <a class="anchor" id="head-8"></a>
 In this section, we run the tlt-infer tool to generate inferences on the trained models.

In [ ]:
# Running inference for detection on n images
# Please go to $USER_EXPERIMENT_DIR/data/faster_rcnn/inference_results_imgs_retrain to see the visualizatons.
!tlt-infer faster_rcnn -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt

The `tlt-infer` tool produces two outputs. 
1. Overlain images in `$USER_EXPERIMENT_DIR/data/faster_rcnn/inference_results_imgs_retrain`
2. Frame by frame bbox labels in kitti format located in `$USER_EXPERIMENT_DIR/data/faster_rcnn/inference_dump_labels_retrain`

In [ ]:
# Simple grid visualizer
%matplotlib inline
import matplotlib.pyplot as plt
import os
from math import ceil
valid_image_ext = ['.jpg', '.png', '.jpeg', '.ppm']

def visualize_images(image_dir, num_cols=4, num_images=10):
    output_path = os.path.join(os.environ['USER_EXPERIMENT_DIR'], image_dir)
    num_rows = int(ceil(float(num_images) / float(num_cols)))
    f, axarr = plt.subplots(num_rows, num_cols, figsize=[80,30])
    f.tight_layout()
    a = [os.path.join(output_path, image) for image in os.listdir(output_path) 
         if os.path.splitext(image)[1].lower() in valid_image_ext]
    for idx, img_path in enumerate(a[:num_images]):
        col_id = idx % num_cols
        row_id = idx / num_cols
        img = plt.imread(img_path)
        axarr[row_id, col_id].imshow(img) 

In [ ]:
# Visualizing the sample images.
OUTPUT_PATH = 'data/faster_rcnn/inference_results_imgs_retrain' # relative path from $USER_EXPERIMENT_DIR.
COLS = 3 # number of columns in the visualizer grid.
IMAGES = 9 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)

 ## 9. Deploy! <a class="anchor" id="head-9"></a>

In [ ]:
# Export in FP32 mode. \
!tlt-export faster_rcnn -m $USER_EXPERIMENT_DIR/data/faster_rcnn/frcnn_kitti_resnet18_retrain.epoch12.tlt  \
                        -o $USER_EXPERIMENT_DIR/data/faster_rcnn/frcnn_kitti_resnet18_retrain.etlt \
                        -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt \
                        -k $KEY

In [ ]:
# Export in FP16 mode. \
# Note that the .etlt model in FP16 mode is  \
# the same as in FP32 mode. \
!tlt-export faster_rcnn -m $USER_EXPERIMENT_DIR/data/faster_rcnn/frcnn_kitti_resnet18_retrain.epoch12.tlt  \
                        -o $USER_EXPERIMENT_DIR/data/faster_rcnn/frcnn_kitti_resnet18_retrain_fp16.etlt \
                        -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt \
                        -k $KEY \
                        --data_type fp16

In [ ]:
# Export in INT8 mode(generate calibration cache file). \
# Note that the .etlt model in INT8 mode is the same as \
# in FP32 mode. \
!tlt-export faster_rcnn -m $USER_EXPERIMENT_DIR/data/faster_rcnn/frcnn_kitti_resnet18_retrain.epoch12.tlt  \
                        -o $USER_EXPERIMENT_DIR/data/faster_rcnn/frcnn_kitti_resnet18_retrain_int8.etlt \
                        -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt \
                        -k $KEY \
                        --cal_image_dir  $USER_EXPERIMENT_DIR/data/testing/image_2 \
                        --data_type int8 \
                        --batch_size 8 \
                        --batches 10 \
                        --cal_cache_file $USER_EXPERIMENT_DIR/data/faster_rcnn/cal.bin  \
                        --cal_data_file $USER_EXPERIMENT_DIR/data/faster_rcnn/cal.tensorfile

In [ ]:
# Converting to TensorRT engine(FP32) is omitted here as this is trivial.
# Convert to TensorRT engine(FP16).
# Specify the GPU ID when generating the TensorRT engine and do inference,
# in case there are different GPU types on the machine.
# Make sure your GPU type supports the FP16 data type before running this cell.
%env CUDA_VISIBLE_DEVICES=0
!tlt-converter -k $KEY  \
               -d 3,384,1248 \
               -o dense_class_td/Softmax,dense_regress_td/BiasAdd,proposal \
               -e $USER_EXPERIMENT_DIR/data/faster_rcnn/trt.fp16.engine \
               -m 4 \
               -t fp16 \
               -i nchw \
               $USER_EXPERIMENT_DIR/data/faster_rcnn/frcnn_kitti_resnet18_retrain_fp16.etlt \

In [ ]:
# Convert to TensorRT engine(INT8).
# Specify the GPU ID when generating the TensorRT engine and do inference,
# in case there are different GPU types on the machine.
# Make sure your GPU type supports the INT8 data type before running this cell.
%env CUDA_VISIBLE_DEVICES=0
!tlt-converter -k $KEY  \
               -d 3,384,1248 \
               -o dense_class_td/Softmax,dense_regress_td/BiasAdd,proposal \
               -c $USER_EXPERIMENT_DIR/data/faster_rcnn/cal.bin \
               -e $USER_EXPERIMENT_DIR/data/faster_rcnn/trt.int8.engine \
               -b 8 \
               -m 4 \
               -t int8 \
               -i nchw \
               $USER_EXPERIMENT_DIR/data/faster_rcnn/frcnn_kitti_resnet18_retrain_int8.etlt \

In [ ]:
print('Exported model and converted TensorRT engine:')
print('------------')
!ls -lh $USER_EXPERIMENT_DIR/data/faster_rcnn

In [ ]:
# Do inference with TensorRT on the generated TensorRT engine
# Please go to $USER_EXPERIMENT_DIR/data/faster_rcnn/inference_results_imgs_retrain to see the visualizatons.
# Here we use the INT8 engine for inference, if you want to use FP16 engine instead please
# customize the 'trt_engine' parameter in the spec file below to point to the FP16 engine.
!sed -i s/#trt/trt/g $SPECS_DIR/default_spec_resnet18_retrain_spec.txt
!sed -i s/#}/}/g $SPECS_DIR/default_spec_resnet18_retrain_spec.txt
!tlt-infer faster_rcnn -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt

The `tlt-infer` tool produces two outputs. 
The paths to the two outputs are exactly the same as the first `tlt-infer` command.

In [ ]:
# Visualizing the sample images from TensorRT inference.
OUTPUT_PATH = 'data/faster_rcnn/inference_results_imgs_retrain' # relative path from $USER_EXPERIMENT_DIR.
COLS = 3 # number of columns in the visualizer grid.
IMAGES = 9 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)